In [19]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from bokeh.plotting import figure, show, output_file, output_notebook
from bokeh.palettes import Spectral11, colorblind, Inferno, BuGn, brewer
from bokeh.models import Span, Title, HoverTool, FixedTicker, value, Label, Legend, Div, ColumnDataSource,LinearColorMapper,Jitter, ColorBar, Range1d,LinearAxis
from bokeh.io import export_png
from bokeh.layouts import grid, column
import math
import time
import datetime
from datetime import datetime

In [20]:
#importing data
googlemobility = pd.read_csv('https://www.gstatic.com/covid19/mobility/Global_Mobility_Report.csv')
stringency = pd.read_csv('https://github.com/OxCGRT/covid-policy-tracker/raw/master/data/OxCGRT_latest.csv')

In [21]:
#dropping data points for sub-regions within countries
googlemobility = googlemobility[pd.isna(googlemobility['sub_region_1'])]
#converting to datetime format
googlemobility['date'] = [datetime.strptime(str(x), '%Y-%m-%d') for x in googlemobility['date']]
stringency['Date'] = [datetime.strptime(str(x), '%Y%m%d') for x in stringency['Date']]


In [22]:
#selecting relevant attributes
googlemobility = googlemobility.drop(columns = ['country_region_code', 'sub_region_1', 'sub_region_2'])
stringency = stringency[['CountryName', 'Date','StringencyIndexForDisplay']]
#renaming columns
googlemobility = googlemobility.rename(columns={"country_region": "country", 'retail_and_recreation_percent_change_from_baseline':'retail', 'grocery_and_pharmacy_percent_change_from_baseline': 'grocery', 'parks_percent_change_from_baseline': 'parks', 'transit_stations_percent_change_from_baseline': 'transit', 'workplaces_percent_change_from_baseline':'workplaces', 'residential_percent_change_from_baseline': 'residential'})
stringency = stringency.rename(columns = {'CountryName':'country', 'Date':'date', 'StringencyIndexForDisplay': 'stringency_index'})
#selecting countries for analysis
countries = ['Belgium', 'France', 'Germany', 'Italy', 'Netherlands', 'Spain', 'Switzerland', 'United Kingdom']
stringency = stringency[stringency['country'].isin(countries)]
googlemobility = googlemobility[googlemobility['country'].isin(countries)]

In [23]:
#merging google and stringency data
final = pd.merge(googlemobility, stringency,  how='inner', on = ['country','date'])

In [24]:
#lockdown start dates (Switzerland did not impose lockdown, hence an arbitrary date)
start_dates = ['17/03/2020', '16/03/2020', '20/03/2020', '9/03/2020', '23/03/2020', '14/03/2020', '31/12/2050', '23/03/2020']
#colors for plots
colors = ["blue", "orange", "green", "red", "magenta", "purple", "black", "gold"]
#categories of places
#places = ['retail', 'grocery', 'parks', 'transit', 'workplaces', 'residential']
places = ['workplaces', 'residential', 'parks']

In [25]:
#freedom of association
final['freedom_index'] = (100 - final['stringency_index'])

In [27]:
#separate plot for each country in the list
TOOLS = 'crosshair,save,pan,box_zoom,reset,wheel_zoom'
for j, country in enumerate(countries):
    title = country
    p = figure(title=title, title_location="above", y_axis_type="linear",x_axis_type='datetime', tools=TOOLS, toolbar_location='above', plot_width=800, plot_height=600)
    
    p.yaxis.bounds = (0, 100)
    p.title.text_font_size = '14pt'
    
    #adding second y-axis for Mobility
    p.extra_y_ranges = {"s": Range1d(start=-100, end=100)}
    p.y_range=Range1d(0, 200)
    p.add_layout(LinearAxis(y_range_name="s"), 'right')
    
    #formatting x-axis labels
    p.xaxis.axis_label = 'Date'
    p.xaxis.axis_label_text_font = "calibri"
    p.xaxis.major_label_orientation = math.pi/2
    
    #drawing lines
    for i, place in enumerate(places):
        data = final[final['country'] == country]
        p.line(x=data.date, y=data[place],  line_color=colors[i], line_width=1, y_range_name="s", legend_label = place)
    p.line(x=data.date, y=data.freedom_index,  line_color=colors[i+1], line_width=3 ,legend_label='Freedom Index')
    
    #adding sundays as ticks on x-axis
    sundays = ['16/02/2020', '23/02/2020', '01/03/2020', '08/03/2020', '15/03/2020', '22/03/2020', '29/03/2020', '05/04/2020', '12/04/2020', '19/04/2020', '26/04/2020', '03/05/2020', '10/05/2020', '17/05/2020']
    x = pd.to_datetime(sundays, format = '%d/%m/%Y')
    tick_vals = pd.to_datetime(x).astype(int)/10**6
    p.xaxis.ticker = FixedTicker(ticks=list(tick_vals))
    
    #line indicating start of lockdown
    start_date = time.mktime(pd.to_datetime(start_dates[j],format='%d/%m/%Y').timetuple())*1000
    lockdown_start = Span(location=start_date, dimension='height', line_color='red',line_dash='dashed', line_width=2)
    p.add_layout(lockdown_start)
    
    #label indicating date of lockdown
    date_str = ' Lockdown from ' + pd.to_datetime(start_dates[j],format='%d/%m/%Y').strftime("%d/%m/%Y")
    lockdown_date = Label(x=start_date, y= 85, y_range_name="s", text=date_str, text_color = 'red', text_font_size = '9pt')
    p.add_layout(lockdown_date)
    
    #y-axis labels
    p.add_layout(Title(text='Freedom Index', align="left", text_font = 'calibri', text_font_style = 'italic', text_font_size = '12pt', offset = 10), "left")
    p.add_layout(Title(text='Mobility', align="center",  text_font = 'calibri', text_font_style = 'italic', text_font_size = '12pt'), "right")
    
    #footnotes
    label_opts = dict(x=0, y=0, x_units='screen', y_units='screen')
    caption1 = Label(text='Data Sources:', **label_opts, text_font_size='8pt', text_font_style = 'italic')
    caption2 = Label(text='Oxford COVID-19 Government Response Tracker, Blavatnik School of Government(accessed-28/05/2020)', **label_opts, text_font_size='8pt', text_font_style = 'italic')
    caption3 = Label(text='Google COVID-19 Community Mobility Reports(https://www.google.com/covid19/mobility/, accessed-28/05/2020)', **label_opts, text_font_size='8pt', text_font_style = 'italic')
    p.add_layout(caption1, 'below')
    p.add_layout(caption2, 'below')
    p.add_layout(caption3, 'below')
 
    p.legend.location = "top_left"
    p.legend.background_fill_alpha = 0.4
        
    show(p)
